In [1]:
import pandas as pd
import numpy as np
import os
import json
from haversine import haversine
# import zipcodes
import datetime
from multiprocessing import Pool
os.getcwd()

'/home/jian/Projects/Savers/TA_creation'

In [2]:
df_CA_zip_centers=pd.read_csv("/home/jian/Docs/Geo_mapping/Canadian_Postal_Codes/canada-postal-code-database-BUSINESS-CSV/canada-postal-code-database-BUSINESS.csv",
                              dtype=str,usecols=["PostalCode",'Latitude','Longitude']).drop_duplicates()

df_CA_zip_centers['Latitude']=df_CA_zip_centers['Latitude'].astype(float)
df_CA_zip_centers['Longitude']=df_CA_zip_centers['Longitude'].astype(float)


df_CA_zip_centers=df_CA_zip_centers.append(pd.DataFrame({"PostalCode":"T1Y 6J6","Latitude":51.0827776,"Longitude":-113.9868132},index=[len(df_CA_zip_centers)+1]))
df_CA_zip_centers=df_CA_zip_centers.append(pd.DataFrame({"PostalCode":"N3S 7P2","Latitude":43.1687593,"Longitude":-80.249613},index=[len(df_CA_zip_centers)+2]))
df_CA_zip_centers=df_CA_zip_centers.append(pd.DataFrame({"PostalCode":"L0R 2H2","Latitude":43.3166868,"Longitude":-79.9187285},index=[len(df_CA_zip_centers)+3]))



In [3]:
df_CA_zip_centers['zip_centers']=df_CA_zip_centers[['Latitude', 'Longitude']].values.tolist()
zip_centers=df_CA_zip_centers.set_index(['PostalCode']).to_dict()['zip_centers']

In [4]:
stroe_list_QCed=pd.read_csv("/home/jian/Projects/Savers/TA_creation/store_data/Savers_zips_by_store_for_CA_stores_JL_2019-03-19.csv",dtype=str,sep="|")
zips_by_store=pd.read_csv("/home/jian/Projects/Savers/TA_creation/store_data/Savers_by_zips_for_CA_stores_JL_2019-03-19.csv",dtype=str,sep="|")

# Actually not QCed for the CA stores because using the store zip

In [5]:
store_cluster_dict=stroe_list_QCed.set_index("location_id").to_dict()['Cluster']
store_cluster_dict.update({"2020":"3"})
store_cluster_dict.update({"2068":"1"})
store_cluster_dict.update({"2096":"3"})
store_cluster_dict.update({"2123":"3"})
store_cluster_dict.update({"2152":"3"})

stroe_list_QCed['Cluster']=stroe_list_QCed['location_id'].apply(lambda x: store_cluster_dict[x])
zips_by_store['Cluster']=zips_by_store['location_id'].apply(lambda x: store_cluster_dict[x])

In [6]:
zips_by_store=zips_by_store.sort_values('location_id').reset_index()
del zips_by_store['index']
stroe_list_QCed=stroe_list_QCed.sort_values('location_id').reset_index()
del stroe_list_QCed['index']

stroe_list_QCed['zip_list_in_5_miles']=stroe_list_QCed['zip_list_in_5_miles'].apply(lambda x: list(eval(x)[0]))

In [7]:
store_zips_dict=stroe_list_QCed.set_index(['location_id']).to_dict()['store_zip']

In [8]:
df_TA_zips=pd.DataFrame({"location_id":[stroe_list_QCed['location_id'][0]]*len(stroe_list_QCed['zip_list_in_5_miles'][0]),"zip_cd":stroe_list_QCed['zip_list_in_5_miles'][0],"TA":[1]*len(stroe_list_QCed['zip_list_in_5_miles'][0])},index=[1]*len(stroe_list_QCed['zip_list_in_5_miles'][0]))

TA_counter=1

for i in range(1,len(stroe_list_QCed)):
    intersection_zip=list(set(stroe_list_QCed['zip_list_in_5_miles'][i]).intersection(set(df_TA_zips['zip_cd'].unique().tolist())))
    if len(intersection_zip)==0:
        TA_counter+=1
        df_TA_zips=df_TA_zips.append(pd.DataFrame({"location_id":[stroe_list_QCed['location_id'][i]]*len(stroe_list_QCed['zip_list_in_5_miles'][i]),"zip_cd":stroe_list_QCed['zip_list_in_5_miles'][i],"TA":[TA_counter]*len(stroe_list_QCed['zip_list_in_5_miles'][i])},index=[i]*len(stroe_list_QCed['zip_list_in_5_miles'][i]))).drop_duplicates()
        
    else:
        df_intersection=df_TA_zips[df_TA_zips['zip_cd'].isin(intersection_zip)]
        group_df_intersection=df_intersection.groupby(['TA'])['zip_cd'].count().to_frame().reset_index().sort_values(['zip_cd'],ascending=False)
        selected_TA=group_df_intersection['TA'][0] 
        
        df_TA_zips_0=df_TA_zips[~df_TA_zips['TA'].isin(set(group_df_intersection['TA']))]
        df_TA_zips_1=df_TA_zips[df_TA_zips['TA'].isin(group_df_intersection['TA'].tolist())]
        df_TA_zips_1['TA']=selected_TA
        df_TA_zips=df_TA_zips_0.append(df_TA_zips_1).append(pd.DataFrame({"location_id":[stroe_list_QCed['location_id'][i]]*len(stroe_list_QCed['zip_list_in_5_miles'][i]),"zip_cd":stroe_list_QCed['zip_list_in_5_miles'][i],"TA":[selected_TA]*len(stroe_list_QCed['zip_list_in_5_miles'][i])},index=[i]*len(stroe_list_QCed['zip_list_in_5_miles'][i]))).drop_duplicates()
        
# dict_TA_zips=df_TA_zips.set_index('zip_cd').to_dict()['TA']
dict_TA_store=df_TA_zips.set_index('location_id').to_dict()['TA']
stroe_list_QCed['TA']=stroe_list_QCed['location_id'].apply(lambda x: dict_TA_store[x])

del dict_TA_store

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [9]:
store_cluster=stroe_list_QCed[['location_id','Cluster']].drop_duplicates()

df_TA_zips=pd.merge(df_TA_zips,store_cluster,on="location_id",how="left")
df_TA_zips.head(2)

,TA,location_id,zip_cd,Cluster
0,2,2004,V1Y 8C8,2
1,2,2004,V1Y 5T5,2


In [10]:
def count_unique(x):
    return len(set(x))

test=df_TA_zips.groupby(['TA'])['Cluster'].apply(count_unique).to_frame().reset_index().sort_values("Cluster",ascending=False)

print(test['Cluster'].unique())
test.head(3) # 1 TA only covers 1 cluster

[1]


,TA,Cluster
0,1,1
43,49,1
49,55,1


In [11]:
summary_store=df_TA_zips.groupby("TA")['location_id'].apply(count_unique).to_frame().reset_index().rename(columns={"location_id":"store_count"})
summary_zip=df_TA_zips[['TA','zip_cd']].drop_duplicates().groupby("TA")['zip_cd'].apply(count_unique).to_frame().reset_index().rename(columns={"zip_cd":"zip_count"})

summary_store_list=df_TA_zips.groupby("TA")['location_id'].apply(set).to_frame().reset_index().rename(columns={"storenumber":"store_list"})
summary_zip_list=df_TA_zips[['TA','zip_cd']].drop_duplicates().groupby("TA")['zip_cd'].apply(list).to_frame().reset_index().rename(columns={"zip_cd":"zip_list"})

summary_by_TA=pd.merge(summary_store,summary_zip,on="TA",how="outer")
summary_by_TA=pd.merge(summary_by_TA,summary_store_list,on="TA",how="outer")
summary_by_TA=pd.merge(summary_by_TA,summary_zip_list,on="TA",how="outer")

'''
Good to use the original store zips

TA_Store_zip_list=data.groupby(['TA'])['revised_store_zip'].apply(set).to_frame().reset_index().rename(columns={"revised_store_zip":"store_zip_list"})
summary_by_TA=pd.merge(summary_by_TA,TA_Store_zip_list,on="TA",how="left")
'''
summary_by_TA=pd.merge(summary_by_TA,df_TA_zips[['TA','Cluster']].drop_duplicates(),on="TA",how="outer")
summary_by_TA=summary_by_TA.reset_index()
del summary_by_TA['index']
summary_by_TA=summary_by_TA.reset_index().rename(columns={"index":"ta_number"})
summary_by_TA['ta_number']=summary_by_TA['ta_number']+1

TA_rename_dict=summary_by_TA.set_index("TA").to_dict()['ta_number']
del summary_by_TA['TA']

In [12]:
df_TA_zips['ta_number']=df_TA_zips['TA'].apply(lambda x: TA_rename_dict[x])
del df_TA_zips['TA']

stroe_list_QCed['ta_number']=stroe_list_QCed['TA'].apply(lambda x: TA_rename_dict[x])
del stroe_list_QCed['TA']

dict_TA_zips=df_TA_zips.set_index('zip_cd').to_dict()['ta_number']
dict_TA_store=df_TA_zips.set_index('location_id').to_dict()['ta_number']

In [13]:
summary_by_store_count=summary_by_TA.groupby(['store_count'])['ta_number'].count().to_frame().reset_index().rename(columns={"ta_number":"TA_count"})
summary_by_store_list=summary_by_TA.groupby(['store_count'])['ta_number'].apply(list).to_frame().reset_index().rename(columns={"ta_number":"TA_list"})
summary_by_store_count=pd.merge(summary_by_store_count,summary_by_store_list,on="store_count",how="outer")
summary_by_store_count

,store_count,TA_count,TA_list
0,1,52,"[2, 5, 9, 14, 16, 17, 19, 21, 22, 25, 26, 27, ..."
1,2,6,"[8, 10, 12, 23, 24, 41]"
2,3,1,[18]
3,4,2,"[4, 15]"
4,5,1,[7]
5,6,1,[11]
6,7,2,"[3, 20]"
7,10,2,"[1, 6]"
8,23,1,[13]


In [14]:
def appending_dist_in_TA(df):
    df=df.reset_index()
    del df['index']
    
    df['dist_max']=np.nan
    df['dist_median']=np.nan
    df['dist_mean']=np.nan
    df['dist_min']=np.nan
    df['max_dist_zip']=np.nan
    
    for index,row in df.iterrows():
        zip_list=row['zip_list']
        store_list=row['location_id']
        dist_list=[]
        maxium_dist=0
        maxium_dist_zip=np.nan
        
        # print("store started" + str(index))
        for store in store_list:
            store_zip=store_zips_dict[store]
            for TA_zip in zip_list:
                dist=haversine(zip_centers[TA_zip],zip_centers[store_zip],miles=True)
                dist_list.append(dist)
                if dist>maxium_dist:
                    maxium_dist_zip=TA_zip
                    maxium_dist=dist
                
        if len(dist_list)>=1:
            df.loc[index,'dist_max']=maxium_dist
            df.loc[index,'dist_median']=np.median(dist_list)
            df.loc[index,'dist_mean']=np.mean(dist_list)
            df.loc[index,'dist_min']=np.min(dist_list)
            df.loc[index,'max_dist_zip']=TA_zip
    
    return df

In [15]:

summary_by_TA=appending_dist_in_TA(summary_by_TA)
summary_by_TA['ratio_max_to_median']=summary_by_TA['dist_max']/summary_by_TA['dist_median']


In [16]:
'''
# 68 in the total
processors=24

summary_by_TA['dist_max']=np.nan
summary_by_TA['dist_median']=np.nan
summary_by_TA['dist_mean']=np.nan
summary_by_TA['dist_min']=np.nan
summary_by_TA=summary_by_TA.sort_values("zip_count",ascending=False)
summary_by_TA=summary_by_TA.reset_index()
del summary_by_TA['index']

multiply_processing_df_input_list=[]
interval=int(np.ceil(len(summary_by_TA)/processors))
for i in range(processors):
    row_selected=[]
    for j in range(interval):
        if i+j*processors<=len(summary_by_TA)-1:
            row_selected=row_selected+[i+j*processors]
    
    locals()['list_df_'+str(i)]=summary_by_TA.iloc[row_selected,:]
    multiply_processing_df_input_list=multiply_processing_df_input_list+[locals()['list_df_'+str(i)]]

print(len(multiply_processing_df_input_list))
'''

'\n# 68 in the total\nprocessors=24\n\nsummary_by_TA[\'dist_max\']=np.nan\nsummary_by_TA[\'dist_median\']=np.nan\nsummary_by_TA[\'dist_mean\']=np.nan\nsummary_by_TA[\'dist_min\']=np.nan\nsummary_by_TA=summary_by_TA.sort_values("zip_count",ascending=False)\nsummary_by_TA=summary_by_TA.reset_index()\ndel summary_by_TA[\'index\']\n\nmultiply_processing_df_input_list=[]\ninterval=int(np.ceil(len(summary_by_TA)/processors))\nfor i in range(processors):\n    row_selected=[]\n    for j in range(interval):\n        if i+j*processors<=len(summary_by_TA)-1:\n            row_selected=row_selected+[i+j*processors]\n    \n    locals()[\'list_df_\'+str(i)]=summary_by_TA.iloc[row_selected,:]\n    multiply_processing_df_input_list=multiply_processing_df_input_list+[locals()[\'list_df_\'+str(i)]]\n\nprint(len(multiply_processing_df_input_list))\n'

In [17]:
'''
summary_by_TA=pd.DataFrame()
print("MP started:"+str(datetime.datetime.now()))
if __name__ == '__main__':
    k=0
    p = Pool(processors)
    result=p.map(appending_dist_in_TA, multiply_processing_df_input_list)
    for res in result:
        if res is not None:
            summary_by_TA=summary_by_TA.append(res)
            k+=1
            print(k,datetime.datetime.now())
    p.close()
    p.join()
    
    

summary_by_TA['ratio_max_to_median']=summary_by_TA['dist_max']/summary_by_TA['dist_median']

print("MP ended:"+str(datetime.datetime.now()))
'''

'\nsummary_by_TA=pd.DataFrame()\nprint("MP started:"+str(datetime.datetime.now()))\nif __name__ == \'__main__\':\n    k=0\n    p = Pool(processors)\n    result=p.map(appending_dist_in_TA, multiply_processing_df_input_list)\n    for res in result:\n        if res is not None:\n            summary_by_TA=summary_by_TA.append(res)\n            k+=1\n            print(k,datetime.datetime.now())\n    p.close()\n    p.join()\n    \n    \n\nsummary_by_TA[\'ratio_max_to_median\']=summary_by_TA[\'dist_max\']/summary_by_TA[\'dist_median\']\n\nprint("MP ended:"+str(datetime.datetime.now()))\n'

In [18]:
'''
summary_by_TA_keep=summary_by_TA[(summary_by_TA['dist_median']<=6) | (summary_by_TA['store_count']<2)]
summary_by_TA_revise=summary_by_TA[(summary_by_TA['dist_median']>6) & (summary_by_TA['store_count']>=2)]

summary_by_TA_revise
'''

"\nsummary_by_TA_keep=summary_by_TA[(summary_by_TA['dist_median']<=6) | (summary_by_TA['store_count']<2)]\nsummary_by_TA_revise=summary_by_TA[(summary_by_TA['dist_median']>6) & (summary_by_TA['store_count']>=2)]\n\nsummary_by_TA_revise\n"

In [19]:
'''
summary_by_TA_keep=summary_by_TA[(summary_by_TA['dist_median']<=6) | (summary_by_TA['store_count']<2)]
summary_by_TA_revise=summary_by_TA[(summary_by_TA['dist_median']>6) & (summary_by_TA['store_count']>=2)]
loop_counter=0
while len(summary_by_TA_revise)>0:
    summary_by_TA_revise=summary_by_TA_revise.reset_index()
    del summary_by_TA_revise['index']
    for i in range(len(summary_by_TA_revise)):
        zip_list=summary_by_TA_revise['zip_list'][i]
        zip_to_remove=summary_by_TA_revise['max_dist_zip'][i]
        zip_list.remove(zip_to_remove)
        summary_by_TA_revise.loc[i,'zip_list']=str(zip_list)
    summary_by_TA_revise['zip_list']=summary_by_TA_revise['zip_list'].apply(lambda x: eval(x))    
    summary_by_TA_revise=appending_dist_in_TA(summary_by_TA_revise)
    
    summary_by_TA_keep_new=summary_by_TA_revise[(summary_by_TA_revise['dist_median']<=6) | (summary_by_TA_revise['store_count']<2)]
    summary_by_TA_keep=summary_by_TA_keep.append(summary_by_TA_keep_new)
    
    summary_by_TA_revise=summary_by_TA_revise[(summary_by_TA_revise['dist_median']>6) & (summary_by_TA_revise['store_count']>=2)]
    loop_counter+=1
    if loop_counter%30==2:
        print(loop_counter,datetime.datetime.now(),len(summary_by_TA_revise))
    if loop_counter>=35:
        break
'''

"\nsummary_by_TA_keep=summary_by_TA[(summary_by_TA['dist_median']<=6) | (summary_by_TA['store_count']<2)]\nsummary_by_TA_revise=summary_by_TA[(summary_by_TA['dist_median']>6) & (summary_by_TA['store_count']>=2)]\nloop_counter=0\nwhile len(summary_by_TA_revise)>0:\n    summary_by_TA_revise=summary_by_TA_revise.reset_index()\n    del summary_by_TA_revise['index']\n    for i in range(len(summary_by_TA_revise)):\n        zip_list=summary_by_TA_revise['zip_list'][i]\n        zip_to_remove=summary_by_TA_revise['max_dist_zip'][i]\n        zip_list.remove(zip_to_remove)\n        summary_by_TA_revise.loc[i,'zip_list']=str(zip_list)\n    summary_by_TA_revise['zip_list']=summary_by_TA_revise['zip_list'].apply(lambda x: eval(x))    \n    summary_by_TA_revise=appending_dist_in_TA(summary_by_TA_revise)\n    \n    summary_by_TA_keep_new=summary_by_TA_revise[(summary_by_TA_revise['dist_median']<=6) | (summary_by_TA_revise['store_count']<2)]\n    summary_by_TA_keep=summary_by_TA_keep.append(summary_by_

In [20]:
'''
summary_by_TA_revise=summary_by_TA[(summary_by_TA['dist_median']>6) & (summary_by_TA['store_count']>=2)]
summary_by_TA_revise['dist_max'][19]
'''

"\nsummary_by_TA_revise=summary_by_TA[(summary_by_TA['dist_median']>6) & (summary_by_TA['store_count']>=2)]\nsummary_by_TA_revise['dist_max'][19]\n"

In [21]:

CA_zip_info=pd.read_csv("/home/jian/Docs/Geo_mapping/Canadian_Postal_Codes/canada-postal-code-database-BUSINESS-CSV/canada-postal-code-database-BUSINESS.csv",
                       dtype=str,usecols=['PostalCode','City','Province']).drop_duplicates("PostalCode")
CA_zip_city_dict=CA_zip_info.set_index("PostalCode").to_dict()['City']
CA_zip_state_dict=CA_zip_info.set_index("PostalCode").to_dict()['Province']


In [22]:
df_city_state_in_TA=df_TA_zips[["ta_number","zip_cd"]].drop_duplicates().reset_index()
del df_city_state_in_TA['index']

def city_of_zip(x):
    try:
        city=CA_zip_city_dict[x]
    except:
        city=np.nan
    return city

def state_of_zip(x):
    try:
        state=CA_zip_state_dict[x]
    except:
        state=np.nan
    return state
    
df_city_state_in_TA['city']=df_city_state_in_TA['zip_cd'].apply(lambda x: city_of_zip(x))
df_city_state_in_TA['state']=df_city_state_in_TA['zip_cd'].apply(lambda x: state_of_zip(x))
df_city_state_in_TA['city']=df_city_state_in_TA['city']+" ("+df_city_state_in_TA['state']+")"



In [23]:
df_city_TA_list=df_city_state_in_TA.groupby(['ta_number'])['city'].apply(set).to_frame().reset_index()
df_city_TA_list=df_city_TA_list.rename(columns={"city":"city_list"})
df_state_TA_list=df_city_state_in_TA.groupby(['ta_number'])['state'].apply(set).to_frame().reset_index()
df_state_TA_list=df_state_TA_list.rename(columns={"state":"state_list"})


In [24]:
counter_k=1
df_primary_city_state=pd.DataFrame()
for ta,group in df_city_state_in_TA.groupby(['ta_number']):
    df_city=group.groupby(['city'])['zip_cd'].count().to_frame().reset_index().sort_values(['zip_cd'],ascending=False).reset_index()
    del df_city['index']
    primary_city=df_city['city'][0]
    
    df_state=group.groupby(['state'])['zip_cd'].count().to_frame().reset_index().sort_values(['zip_cd'],ascending=False).reset_index()
    del df_state['index']
    primary_state=df_state['state'][0]
    
    df=pd.DataFrame({"ta_number":ta,"Primary_City":primary_city,"Primary_State":primary_state},index=[counter_k])
    counter_k+=1
    df_primary_city_state=df_primary_city_state.append(df)

In [25]:
summary_by_TA=pd.merge(summary_by_TA,df_city_TA_list,on="ta_number",how="left")
summary_by_TA=pd.merge(summary_by_TA,df_state_TA_list,on="ta_number",how="left")

summary_by_TA=pd.merge(summary_by_TA,df_primary_city_state,on="ta_number",how="left")

summary_by_TA=summary_by_TA.rename(columns={"location_id":"store_list"})
summary_by_TA['ta_number']=summary_by_TA['ta_number'].astype(str)
stroe_list_QCed['ta_number']=stroe_list_QCed['ta_number'].astype(str)
df_TA_zips['ta_number']=df_TA_zips['ta_number'].astype(str)

In [26]:
summary_by_TA_to_write=summary_by_TA.copy()
del summary_by_TA_to_write['zip_list']
summary_by_TA_to_write=summary_by_TA_to_write.sort_values("store_count",ascending=False)

stroe_list_QCed_to_writer=stroe_list_QCed.copy()
del stroe_list_QCed_to_writer['zip_list_in_5_miles']
stroe_list_QCed_to_writer['location_id']=stroe_list_QCed_to_writer['location_id'].astype(int)
stroe_list_QCed_to_writer=stroe_list_QCed_to_writer.sort_values("location_id")


In [27]:
summary_by_TA['store_count']=summary_by_TA['store_count'].astype(str)
summary_by_TA['store_list']=summary_by_TA['store_list'].astype(str)
summary_by_TA['TA_label']=summary_by_TA['store_count']+"_"+summary_by_TA['store_list']
TA_label_dict=summary_by_TA.set_index(['ta_number']).to_dict()['TA_label']
df_TA_zips['TA_label']=df_TA_zips['ta_number'].apply(lambda x: TA_label_dict[x])
stroe_list_QCed['TA_label']=stroe_list_QCed['ta_number'].apply(lambda x: TA_label_dict[x])

In [28]:
store_brand=stroe_list_QCed[['location_id','Brand']]

summary_by_TA['Brand_count']=np.nan
for index,row in summary_by_TA.iterrows():
    store_list=list(eval(row['store_list']))
    df_brand_ta=store_brand[store_brand['location_id'].isin(store_list)]
    df_brand_ta_summary=df_brand_ta.groupby(['Brand'])['location_id'].count().to_dict()
    summary_by_TA.loc[index,'Brand_count']=str(df_brand_ta_summary)

In [37]:
summary_by_TA_to_write=summary_by_TA.copy()
del summary_by_TA_to_write['zip_list']
summary_by_TA_to_write["store_count"]=summary_by_TA_to_write["store_count"].astype(int)
summary_by_TA_to_write=summary_by_TA_to_write.sort_values("store_count",ascending=False)


In [38]:
writer=pd.ExcelWriter("/home/jian/Projects/Savers/TA_creation/CA_TA_output/Before_Splitting/Savers_CA_TA_Final_Version_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")

stroe_list_QCed.to_excel(writer,"Store_list",index=False)
summary_by_TA_to_write.to_excel(writer,"TA_Summary",index=False)
summary_by_store_count.to_excel(writer,"TA_Summary_by_store_count",index=False)
df_TA_zips.to_excel(writer,"TA_zips",index=False)

writer.save()



In [30]:
'''
data_tableau_1=stroe_list_QCed[['location_id','ta_number','TA_label','Latitude','Longitude']]
data_tableau_1['Latitude']=data_tableau_1['Latitude'].astype(float)
data_tableau_1['Longitude']=data_tableau_1['Longitude'].astype(float)

data_tableau_2=df_TA_zips[['ta_number','TA_label','zip_cd']].drop_duplicates()

data_tableau=data_tableau_1.append(data_tableau_2)
data_tableau['Latitude']=data_tableau['Latitude'].fillna(0)
data_tableau['Longitude']=data_tableau['Longitude'].fillna(0)


df_TA_dist_median_count=summary_by_TA[['ta_number','dist_median','store_count']]
data_tableau=pd.merge(data_tableau,df_TA_dist_max_count,on="ta_number",how="left")

data_tableau.to_csv("/home/jian/Projects/Savers/TA_creation/CA_TA_output/Before_Splitting/CA_TA_Tableau.csv",index=False)

# Tableau doesn't support Zip 6, but Zip 3 are fine
'''

'\ndata_tableau_1=stroe_list_QCed[[\'location_id\',\'ta_number\',\'TA_label\',\'Latitude\',\'Longitude\']]\ndata_tableau_1[\'Latitude\']=data_tableau_1[\'Latitude\'].astype(float)\ndata_tableau_1[\'Longitude\']=data_tableau_1[\'Longitude\'].astype(float)\n\ndata_tableau_2=df_TA_zips[[\'ta_number\',\'TA_label\',\'zip_cd\']].drop_duplicates()\n\ndata_tableau=data_tableau_1.append(data_tableau_2)\ndata_tableau[\'Latitude\']=data_tableau[\'Latitude\'].fillna(0)\ndata_tableau[\'Longitude\']=data_tableau[\'Longitude\'].fillna(0)\n\n\ndf_TA_dist_median_count=summary_by_TA[[\'ta_number\',\'dist_median\',\'store_count\']]\ndata_tableau=pd.merge(data_tableau,df_TA_dist_max_count,on="ta_number",how="left")\n\ndata_tableau.to_csv("/home/jian/Projects/Savers/TA_creation/CA_TA_output/Before_Splitting/CA_TA_Tableau.csv",index=False)\n\n# Tableau doesn\'t support Zip 6, but Zip 3 are fine\n'